## CODING QUESTIONS

#### 1) Compute matching SIFT keypoints from a stereo image pair.

#### 2) Compute and display the epipolar lines for both images.

#### 3) Pick any one keypoint in the right image which doesn't have a correct match in the left image, and is not on the corresponding epipolar line. Extract a patch of size 5x5 around this keypoint in the right image. 

#### 4) Match the extracted patch using the sum of squared difference with every patch along the corresponding epipolar line in the left image. 

#### 5) Plot SSD values. Find the matching point with minimum SSD value. Display found matching points in both the images.

## REASONING QUESTIONS

#### 6) Did you find exactly one matching point with lowest SSD or multiple? Is the found matching point the correct one? Explain why.

#### 7) Suppose you found multiple matches in the previous question. What constraint(s) could be imposed on the matching algorithm to find a single match?

#### 8) Suppose you found exactly one match with the lowest SSD, but it is the incorrect one. What constraint(s) could be imposed to find a correct match?